In [ ]:
import geopandas as gpd
import pandas as pd
import requests as r

In [ ]:
buildings = gpd.read_file('buildings.geojson')
services = gpd.read_file('services.geojson')
services_names = pd.read_csv('services_names.csv')
reports = gpd.read_file('reports.geojson')
messages = gpd.read_file('messages.geojson')
blocks = gpd.read_file('blocks.geojson')

In [ ]:
print("Q: На что мне обратить внимание по вопросам стратегии развития города?")

SERVICE_TYPE = services_names[services_names['city_service_type'] == 'Детский сад'].iloc[0].city_service_type_code #определяем название сервисв в городской модели, "kindergartens"
CITY = "saint-petersburg"
VAL_TYPE = "normative" #для некоторых сервисов требуется другой подход к расчету, model

request_provision = {
  "city": CITY,
  "service_types": [
    SERVICE_TYPE
  ],
  "year": 2023,
  "calculation_type": "gravity",
  "user_selection_zone": {},
  "valuation_type": "normative",
  "service_impotancy": [
    1
  ]
}

provision = r.post("http://10.32.1.42:5000/api_v3/get_provision", json = request_provision)

provision_buildings = provision.json()['houses']
for feature in provision_buildings['features']:
    feature['properties']['id'] = feature['id']
provision_services = provision.json()['services']
for feature in provision_services['features']:
    feature['properties']['id'] = feature['id']
provision_buildings = gpd.GeoDataFrame.from_features(provision_buildings, crs=4326).drop(columns=['geometry'])
provision_services = gpd.GeoDataFrame.from_features(provision_services, crs=4326).drop(columns=['geometry', 'city_service_type_code'])
provision_buildings.columns = [f"{SERVICE_TYPE}_{col}" if col != 'id' else col for col in provision_buildings.columns]
provision_buildings['id'] = provision_buildings['id'].astype(int)
provision_services.columns = [f"{SERVICE_TYPE}_{col}" if col != 'id' else col for col in provision_services.columns]
provision_services['id'] = provision_services['id'].astype(int)

q_buildings = buildings.merge(provision_buildings, left_on='building_id', right_on='id', how='left').drop(columns=['id'])
q_services = services.merge(provision_services, on='id', how='left')

MEAN_KINDERGARTEN_PROVISION = int(q_buildings.kindergartens_provison_value.dropna().mean().round(2) * 100)
TARGET_KINDERGARTEN_PROVISION = 100
print(f"A: В сфере образования наблюдается разрыв между плановым показателем в {TARGET_KINDERGARTEN_PROVISION}% обеспеченности местами в детских садах и текущим показателем в {MEAN_KINDERGARTEN_PROVISION}%")

Q: На что мне обратить внимание по вопросам стратегии развития города?
A: В сфере образования наблюдается разрыв между плановым показателем в 100% обеспеченности местами в детских садах и текущим показателем в 35%


In [ ]:
print("Q: Какой на данный момент дефицит мест в детских садах в Московском районе?")

SERVICE_TYPE = "kindergartens"
CITY = "saint-petersburg"
VAL_TYPE = "normative" #для некоторых сервисов требуется другой подход к расчету, model

request_provision = {
  "city": CITY,
  "service_types": [
    SERVICE_TYPE
  ],
  "year": 2023,
  "calculation_type": "gravity",
  "user_selection_zone": {},
  "valuation_type": "normative",
  "service_impotancy": [
    1
  ]
}

provision = r.post("http://10.32.1.42:5000/api_v3/get_provision", json = request_provision)

provision_buildings = provision.json()['houses']
for feature in provision_buildings['features']:
    feature['properties']['id'] = feature['id']
provision_services = provision.json()['services']
for feature in provision_services['features']:
    feature['properties']['id'] = feature['id']
provision_buildings = gpd.GeoDataFrame.from_features(provision_buildings, crs=4326).drop(columns=['geometry'])
provision_services = gpd.GeoDataFrame.from_features(provision_services, crs=4326).drop(columns=['geometry', 'city_service_type_code'])
provision_buildings.columns = [f"{SERVICE_TYPE}_{col}" if col != 'id' else col for col in provision_buildings.columns]
provision_buildings['id'] = provision_buildings['id'].astype(int)
provision_services.columns = [f"{SERVICE_TYPE}_{col}" if col != 'id' else col for col in provision_services.columns]
provision_services['id'] = provision_services['id'].astype(int)

q_buildings = buildings.merge(provision_buildings, left_on='building_id', right_on='id', how='left').drop(columns=['id'])
q_services = services.merge(provision_services, on='id', how='left')

district_buildings = q_buildings[q_buildings['administrative_unit'] == 'Московский район']
NUMBER_OF_PLACES_IN_KINDERGARTENS = int(district_buildings.kindergartens_demand_left.sum())
print(f"A: В Московском районе не хватает {NUMBER_OF_PLACES_IN_KINDERGARTENS} мест в детских садах")


Q: Какой на данный момент дефицит мест в детских садах в Московском районе?
A: В Московском районе не хватает 0 мест в детских садах


In [ ]:
print(f"Q: В каких условиях доступности находится здание по адресу Московский проспект, 73к5?")
CITY = "saint-petersburg"
TARGET_BLOCK_ID = int(buildings[buildings['address'] == 'Санкт-Петербург, Московский проспект, 73к5'].iloc[0].block_id)
request_accesibility = {
    "city":CITY,
    "block_id":TARGET_BLOCK_ID
    }
accesibility = gpd.GeoDataFrame.from_features(r.get("http://10.32.1.65:5000/api/v2/blocks_accessibility/get_accessibility", params = request_accesibility).json(), crs=4326)
MEDIAN_ACCESSIBILITY = int(accesibility.median_time.median())
print(f"A: В среднем до любой точки города из этого места можно добраться за {MEDIAN_ACCESSIBILITY} минут")

Q: В каких условиях доступности находится здание по адресу Московский проспект, 73к5?
A: В среднем до любой точки города из этого места можно добраться за 106 минут


In [ ]:
print(f"Q: В каком районе наиболее активно жалуются на благоустройство?")
problem_df = reports[reports['subcategory'] == 'Благоустройство']
district_counts = problem_df.groupby('district').size()
max_complaint_district = district_counts.idxmax()
print(f"A: {max_complaint_district} район имеет наибольшее количество жалоб на благоустройство")

Q: В каком районе наиболее активно жалуются на благоустройство?
A: Невский район имеет наибольшее количество жалоб на благоустройство


In [ ]:
print(f'Q: В каком муниципалитете складыается проблемная ситуация и проблемы в каких сферах в нём присутсвуют?')
municipality_problems = reports.groupby('municipality')['subcategory'].count()
max_problem_municipality = municipality_problems.idxmax()
top_problems = reports[reports['municipality'] == max_problem_municipality]['subcategory'].value_counts().nlargest(5)
top_problems_str = top_problems.to_string()
top_problems_str = '\n'.join([line for line in top_problems_str.split('\n') if not line.strip().endswith('int64')])
print(f"A: Муниципалитет с наибольшим количеством проблем: {max_problem_municipality}. Главными проблемами, которые упоминают жители являются:\n{top_problems_str}")

Q: В каком муниципалитете складыается проблемная ситуация и проблемы в каких сферах в нём присутсвуют?
A: Муниципалитет с наибольшим количеством проблем: муниципальный округ Ульянка. Главными проблемами, которые упоминают жители являются:
subcategory
Содержание МКД                                             6399
Благоустройство                                            3947
Незаконная информационная и (или) рекламная конструкция     169
Санитарное состояние                                        118
Фасад                                                       109


In [ ]:
print("Q: Какие проблемы в городе не решены в течение месяца?")
messages['date'] = pd.to_datetime(messages['date'])
messages = messages.sort_values(by='date')
grouped = messages.groupby('function')
result = grouped.agg( count=('function', 'size'), first_date=('date', 'first'), last_date=('date', 'last')).reset_index()
result = (result.sort_values(by='count', ascending=False) .reset_index(drop=True) .rename(columns={ 'function': 'Область', 'count': 'Количество упоминаний', 'first_date': 'Первое упоминание', 'last_date': 'Последнее упоминание'}))
print("A: В течение месяца не решены проблемы в следующих областях:")
result.head(5)

Q: Какие проблемы в городе не решены в течение месяца?
A: В течение месяца не решены проблемы в следующих областях:


,Область,Количество упоминаний,Первое упоминание,Последнее упоминание
0,Другое,3262,2024-01-01,2024-01-19
1,Безопасность,1089,2024-01-01,2024-01-19
2,ЖКХ,880,2024-01-01,2024-01-19
3,Благоустройство,747,2024-01-01,2024-01-19
4,Энергетика,678,2024-01-01,2024-01-19
